In [29]:
import pandas as pd
import numpy as np
import os

In [33]:
from google.colab import drive

drive.mount('/content/drive')

CURR_DIR = "/content/drive/My Drive"
DATA_FILE = "profiles.csv"

profiles = pd.read_csv(os.path.join(CURR_DIR, DATA_FILE))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Drop essay and income(missing 80% responses as we disscussed in midterm report)





In [34]:
profiles = profiles.drop(['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9', 'income', 'last_online', 'location'], axis=1)
profiles.head(5)

,age,body_type,diet,drinks,drugs,education,ethnicity,height,job,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,"asian, white",7500.0%,transportation,"doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,white,7000.0%,hospitality / travel,"doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (...",single
2,38,thin,anything,socially,NaN,graduated from masters program,NaN,6800.0%,NaN,NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available
3,23,thin,vegetarian,socially,NaN,working on college/university,white,7100.0%,student,doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,"english, german (poorly)",single
4,29,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",6600.0%,artistic / musical / writer,NaN,straight,likes dogs and likes cats,NaN,m,aquarius,no,english,single


In [35]:
print(profiles.shape)
total_nrows = profiles.shape[0]
total_ncols = profiles.shape[1]

print(profiles.info())
# print(profiles.describe())

(59946, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59946 entries, 0 to 59945
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          59946 non-null  int64  
 1   body_type    54650 non-null  object 
 2   diet         35551 non-null  object 
 3   drinks       56961 non-null  object 
 4   drugs        45866 non-null  object 
 5   education    53318 non-null  object 
 6   ethnicity    54266 non-null  object 
 7   height       59943 non-null  float64
 8   job          51748 non-null  object 
 9   offspring    24385 non-null  object 
 10  orientation  59946 non-null  object 
 11  pets         40025 non-null  object 
 12  religion     39720 non-null  object 
 13  sex          59946 non-null  object 
 14  sign         48890 non-null  object 
 15  smokes       54434 non-null  object 
 16  speaks       59896 non-null  object 
 17  status       59946 non-null  object 
dtypes: float64(1), int64(1), object(16

## Check Missing Data

In [36]:
total_count = profiles.isna().count()
count_na = profiles.isna().sum().sort_values(ascending = False)
pct_na =(profiles.isna().sum()/total_count).sort_values(ascending = False)
pd.options.display.float_format = '{:.1%}'.format
missing_data = pd.concat([count_na, pct_na], axis=1, keys=['Count', 'Percent'])

# Only show columns with more than 5% missing data
missing_data[missing_data['Percent']>=0.05].head(20)

,Count,Percent
offspring,35561,59.3%
diet,24395,40.7%
religion,20226,33.7%
pets,19921,33.2%
drugs,14080,23.5%
sign,11056,18.4%
job,8198,13.7%
education,6628,11.1%
ethnicity,5680,9.5%
smokes,5512,9.2%


In [37]:
profiles = profiles.drop(['offspring'], axis=1)
profiles = profiles.dropna()

In [38]:
print(profiles.shape)
print(profiles.info())

(12012, 17)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12012 entries, 0 to 59944
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          12012 non-null  int64  
 1   body_type    12012 non-null  object 
 2   diet         12012 non-null  object 
 3   drinks       12012 non-null  object 
 4   drugs        12012 non-null  object 
 5   education    12012 non-null  object 
 6   ethnicity    12012 non-null  object 
 7   height       12012 non-null  float64
 8   job          12012 non-null  object 
 9   orientation  12012 non-null  object 
 10  pets         12012 non-null  object 
 11  religion     12012 non-null  object 
 12  sex          12012 non-null  object 
 13  sign         12012 non-null  object 
 14  smokes       12012 non-null  object 
 15  speaks       12012 non-null  object 
 16  status       12012 non-null  object 
dtypes: float64(1), int64(1), object(15)
memory usage: 1.6+ MB
None


## Sample 10000 data point


In [39]:
profiles = profiles.sample(n=10000, random_state=47)
profiles = profiles.reset_index(drop=True)

In [40]:
profiles

,age,body_type,diet,drinks,drugs,education,ethnicity,height,job,orientation,pets,religion,sex,sign,smokes,speaks,status
0,28,skinny,mostly anything,socially,never,working on college/university,white,6500.0%,sales / marketing / biz dev,straight,has dogs,christianity but not too serious about it,f,cancer and it&rsquo;s fun to think about,no,"english (fluently), french (okay)",single
1,45,average,mostly vegetarian,socially,never,graduated from law school,white,6000.0%,law / legal services,straight,likes dogs,judaism but not too serious about it,f,aquarius and it&rsquo;s fun to think about,no,"english (fluently), french (okay)",single
2,23,athletic,mostly anything,socially,never,working on two-year college,white,6900.0%,medicine / health,straight,has dogs and likes cats,catholicism but not too serious about it,m,capricorn and it&rsquo;s fun to think about,no,english (fluently),single
3,34,athletic,mostly vegetarian,rarely,never,graduated from ph.d program,asian,6800.0%,medicine / health,straight,likes dogs,agnosticism and laughing about it,m,libra but it doesn&rsquo;t matter,when drinking,"english (fluently), chinese (poorly)",single
4,23,athletic,mostly anything,socially,never,graduated from college/university,white,6900.0%,artistic / musical / writer,straight,has dogs,atheism,m,cancer but it doesn&rsquo;t matter,no,english,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,20,fit,anything,rarely,sometimes,working on college/university,white,7600.0%,sales / marketing / biz dev,straight,likes dogs and has cats,agnosticism,m,pisces but it doesn&rsquo;t matter,sometimes,english,single
9996,52,fit,mostly anything,often,sometimes,graduated from two-year college,white,6900.0%,science / tech / engineering,straight,likes dogs and has cats,agnosticism but not too serious about it,m,cancer and it&rsquo;s fun to think about,no,english,available
9997,19,average,strictly vegetarian,socially,sometimes,working on college/university,white,6100.0%,student,gay,likes dogs and likes cats,atheism and somewhat serious about it,f,aquarius but it doesn&rsquo;t matter,no,english (fluently),single
9998,24,athletic,mostly anything,socially,never,graduated from space camp,white,6900.0%,executive / management,straight,likes dogs and likes cats,other but not too serious about it,m,aries and it&rsquo;s fun to think about,no,"english (fluently), french (okay)",single


## One-hot Encoding


In [41]:
# header = profiles.columns
categorical_header = np.array(['body_type', 'diet', 'drinks', 'drugs', 'education', 'ethnicity',
       'job', 'orientation', 'pets', 'religion', 'sign', 'smokes', 'speaks', 'status', 'sex'])

In [42]:
encode = pd.get_dummies(profiles[categorical_header])
sex = encode['sex_f']
encode = encode.drop(['sex_f', 'sex_m'], axis=1)

In [43]:
result = pd.concat([encode,sex], axis=1)  
result = result.rename(columns={'sex_f':'sex'}) # 1 - female, 0 - male
result

,body_type_a little extra,body_type_athletic,body_type_average,body_type_curvy,body_type_fit,body_type_full figured,body_type_jacked,body_type_overweight,body_type_rather not say,body_type_skinny,body_type_thin,body_type_used up,diet_anything,diet_halal,diet_kosher,diet_mostly anything,diet_mostly halal,diet_mostly kosher,diet_mostly other,diet_mostly vegan,diet_mostly vegetarian,diet_other,diet_strictly anything,diet_strictly halal,diet_strictly kosher,diet_strictly other,diet_strictly vegan,diet_strictly vegetarian,diet_vegan,diet_vegetarian,drinks_desperately,drinks_not at all,drinks_often,drinks_rarely,drinks_socially,drinks_very often,drugs_never,drugs_often,drugs_sometimes,education_college/university,...,"speaks_english, spanish, french, c++ (okay), japanese (poorly)","speaks_english, spanish, french, lisp, russian","speaks_english, spanish, french, other","speaks_english, spanish, german","speaks_english, spanish, hebrew","speaks_english, spanish, italian","speaks_english, spanish, italian (poorly)","speaks_english, spanish, other","speaks_english, spanish, portuguese, italian","speaks_english, spanish, tagalog, portuguese, japanese","speaks_english, swahili","speaks_english, swahili (poorly)","speaks_english, swedish","speaks_english, swedish (poorly)","speaks_english, tagalog","speaks_english, tagalog (fluently), german, spanish","speaks_english, tagalog (fluently), spanish (okay), farsi (poorly)","speaks_english, tagalog (okay)","speaks_english, tagalog (okay), spanish (poorly)","speaks_english, tagalog (poorly)","speaks_english, tagalog (poorly), c++ (okay)","speaks_english, tagalog, chinese","speaks_english, tagalog, spanish","speaks_english, tamil (poorly), spanish (poorly)","speaks_english, tamil, spanish","speaks_english, thai (okay)","speaks_english, thai, other","speaks_english, turkish (poorly), spanish (okay)","speaks_english, vietnamese","speaks_english, vietnamese (fluently)","speaks_english, vietnamese (fluently), sign language (poorly)","speaks_english, vietnamese (okay)","speaks_english, vietnamese (poorly)","speaks_english, yiddish (poorly)",status_available,status_married,status_seeing someone,status_single,status_unknown,sex
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9996,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9997,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
9998,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [13]:
profiles = profiles.drop(categorical_header, axis=1)
profiles = pd.concat([profiles, result], axis=1)
profiles  

,age,height,body_type_a little extra,body_type_athletic,body_type_average,body_type_curvy,body_type_fit,body_type_full figured,body_type_jacked,body_type_overweight,body_type_rather not say,body_type_skinny,body_type_thin,body_type_used up,diet_anything,diet_halal,diet_kosher,diet_mostly anything,diet_mostly halal,diet_mostly kosher,diet_mostly other,diet_mostly vegan,diet_mostly vegetarian,diet_other,diet_strictly anything,diet_strictly halal,diet_strictly kosher,diet_strictly other,diet_strictly vegan,diet_strictly vegetarian,diet_vegan,diet_vegetarian,drinks_desperately,drinks_not at all,drinks_often,drinks_rarely,drinks_socially,drinks_very often,drugs_never,drugs_often,...,"speaks_english, spanish, french, c++ (okay), japanese (poorly)","speaks_english, spanish, french, lisp, russian","speaks_english, spanish, french, other","speaks_english, spanish, german","speaks_english, spanish, hebrew","speaks_english, spanish, italian","speaks_english, spanish, italian (poorly)","speaks_english, spanish, other","speaks_english, spanish, portuguese, italian","speaks_english, spanish, tagalog, portuguese, japanese","speaks_english, swahili","speaks_english, swahili (poorly)","speaks_english, swedish","speaks_english, swedish (poorly)","speaks_english, tagalog","speaks_english, tagalog (fluently), german, spanish","speaks_english, tagalog (fluently), spanish (okay), farsi (poorly)","speaks_english, tagalog (okay)","speaks_english, tagalog (okay), spanish (poorly)","speaks_english, tagalog (poorly)","speaks_english, tagalog (poorly), c++ (okay)","speaks_english, tagalog, chinese","speaks_english, tagalog, spanish","speaks_english, tamil (poorly), spanish (poorly)","speaks_english, tamil, spanish","speaks_english, thai (okay)","speaks_english, thai, other","speaks_english, turkish (poorly), spanish (okay)","speaks_english, vietnamese","speaks_english, vietnamese (fluently)","speaks_english, vietnamese (fluently), sign language (poorly)","speaks_english, vietnamese (okay)","speaks_english, vietnamese (poorly)","speaks_english, yiddish (poorly)",status_available,status_married,status_seeing someone,status_single,status_unknown,sex
0,28,6500.0%,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,45,6000.0%,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,23,6900.0%,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,34,6800.0%,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,23,6900.0%,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,20,7600.0%,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9996,52,6900.0%,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9997,19,6100.0%,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
9998,24,6900.0%,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Discretize Continuous Value

In [14]:
profiles['height'] = pd.cut(profiles['height'], bins=5, labels=[0,1,2,3,4])
profiles['age'] = pd.cut(profiles['age'], bins=5, labels=[0,1,2,3,4])

In [15]:
height = profiles['height'].unique()
age = profiles['age'].unique()
print(height)
print(age)

[2, 3, 1, 4, 0]
Categories (5, int64): [0 < 1 < 2 < 3 < 4]
[0, 2, 1, 3, 4]
Categories (5, int64): [0 < 1 < 2 < 3 < 4]


In [16]:
profiles['height'] = pd.cut(profiles['height'], bins=5, labels=[0,1,2,3,4])
profiles['age'] = pd.cut(profiles['age'], bins=5, labels=[0,1,2,3,4])
profiles

,age,height,body_type_a little extra,body_type_athletic,body_type_average,body_type_curvy,body_type_fit,body_type_full figured,body_type_jacked,body_type_overweight,body_type_rather not say,body_type_skinny,body_type_thin,body_type_used up,diet_anything,diet_halal,diet_kosher,diet_mostly anything,diet_mostly halal,diet_mostly kosher,diet_mostly other,diet_mostly vegan,diet_mostly vegetarian,diet_other,diet_strictly anything,diet_strictly halal,diet_strictly kosher,diet_strictly other,diet_strictly vegan,diet_strictly vegetarian,diet_vegan,diet_vegetarian,drinks_desperately,drinks_not at all,drinks_often,drinks_rarely,drinks_socially,drinks_very often,drugs_never,drugs_often,...,"speaks_english, spanish, french, c++ (okay), japanese (poorly)","speaks_english, spanish, french, lisp, russian","speaks_english, spanish, french, other","speaks_english, spanish, german","speaks_english, spanish, hebrew","speaks_english, spanish, italian","speaks_english, spanish, italian (poorly)","speaks_english, spanish, other","speaks_english, spanish, portuguese, italian","speaks_english, spanish, tagalog, portuguese, japanese","speaks_english, swahili","speaks_english, swahili (poorly)","speaks_english, swedish","speaks_english, swedish (poorly)","speaks_english, tagalog","speaks_english, tagalog (fluently), german, spanish","speaks_english, tagalog (fluently), spanish (okay), farsi (poorly)","speaks_english, tagalog (okay)","speaks_english, tagalog (okay), spanish (poorly)","speaks_english, tagalog (poorly)","speaks_english, tagalog (poorly), c++ (okay)","speaks_english, tagalog, chinese","speaks_english, tagalog, spanish","speaks_english, tamil (poorly), spanish (poorly)","speaks_english, tamil, spanish","speaks_english, thai (okay)","speaks_english, thai, other","speaks_english, turkish (poorly), spanish (okay)","speaks_english, vietnamese","speaks_english, vietnamese (fluently)","speaks_english, vietnamese (fluently), sign language (poorly)","speaks_english, vietnamese (okay)","speaks_english, vietnamese (poorly)","speaks_english, yiddish (poorly)",status_available,status_married,status_seeing someone,status_single,status_unknown,sex
0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,2,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,3,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9996,3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9997,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
9998,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


## Generate TrainingSet and TestSet






In [23]:
profiles.to_csv('profiles_full.csv', index=None)

In [28]:
# Check 
CURR_DIR = "/content/drive/My Drive"
DATA_FILE = "profiles_full.csv"

full_df = pd.read_csv(os.path.join(CURR_DIR, DATA_FILE))
full_df

,age,height,body_type_a little extra,body_type_athletic,body_type_average,body_type_curvy,body_type_fit,body_type_full figured,body_type_jacked,body_type_overweight,body_type_rather not say,body_type_skinny,body_type_thin,body_type_used up,diet_anything,diet_halal,diet_kosher,diet_mostly anything,diet_mostly halal,diet_mostly kosher,diet_mostly other,diet_mostly vegan,diet_mostly vegetarian,diet_other,diet_strictly anything,diet_strictly halal,diet_strictly kosher,diet_strictly other,diet_strictly vegan,diet_strictly vegetarian,diet_vegan,diet_vegetarian,drinks_desperately,drinks_not at all,drinks_often,drinks_rarely,drinks_socially,drinks_very often,drugs_never,drugs_often,...,"speaks_english, spanish, french, c++ (okay), japanese (poorly)","speaks_english, spanish, french, lisp, russian","speaks_english, spanish, french, other","speaks_english, spanish, german","speaks_english, spanish, hebrew","speaks_english, spanish, italian","speaks_english, spanish, italian (poorly)","speaks_english, spanish, other","speaks_english, spanish, portuguese, italian","speaks_english, spanish, tagalog, portuguese, japanese","speaks_english, swahili","speaks_english, swahili (poorly)","speaks_english, swedish","speaks_english, swedish (poorly)","speaks_english, tagalog","speaks_english, tagalog (fluently), german, spanish","speaks_english, tagalog (fluently), spanish (okay), farsi (poorly)","speaks_english, tagalog (okay)","speaks_english, tagalog (okay), spanish (poorly)","speaks_english, tagalog (poorly)","speaks_english, tagalog (poorly), c++ (okay)","speaks_english, tagalog, chinese","speaks_english, tagalog, spanish","speaks_english, tamil (poorly), spanish (poorly)","speaks_english, tamil, spanish","speaks_english, thai (okay)","speaks_english, thai, other","speaks_english, turkish (poorly), spanish (okay)","speaks_english, vietnamese","speaks_english, vietnamese (fluently)","speaks_english, vietnamese (fluently), sign language (poorly)","speaks_english, vietnamese (okay)","speaks_english, vietnamese (poorly)","speaks_english, yiddish (poorly)",status_available,status_married,status_seeing someone,status_single,status_unknown,sex
0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,2,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,3,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9996,3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9997,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
9998,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [21]:
# 1.5 Generate TrainingSet and TestSet

# Sampling
test_df = profiles.sample(frac=0.2, random_state=47)
train_df = profiles[~profiles.index.isin(test_df.index)]

# Drop Index
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

# Save Files
test_df.to_csv('testSet.csv', index=None)
train_df.to_csv('trainingSet.csv', index=None)

In [22]:
# Check 
CURR_DIR = "/content/drive/My Drive"
DATA_FILE = "trainingSet.csv"

train_df = pd.read_csv(os.path.join(CURR_DIR, DATA_FILE))
train_df

,age,height,body_type_a little extra,body_type_athletic,body_type_average,body_type_curvy,body_type_fit,body_type_full figured,body_type_jacked,body_type_overweight,body_type_rather not say,body_type_skinny,body_type_thin,body_type_used up,diet_anything,diet_halal,diet_kosher,diet_mostly anything,diet_mostly halal,diet_mostly kosher,diet_mostly other,diet_mostly vegan,diet_mostly vegetarian,diet_other,diet_strictly anything,diet_strictly halal,diet_strictly kosher,diet_strictly other,diet_strictly vegan,diet_strictly vegetarian,diet_vegan,diet_vegetarian,drinks_desperately,drinks_not at all,drinks_often,drinks_rarely,drinks_socially,drinks_very often,drugs_never,drugs_often,...,"speaks_english, spanish, french, c++ (okay), japanese (poorly)","speaks_english, spanish, french, lisp, russian","speaks_english, spanish, french, other","speaks_english, spanish, german","speaks_english, spanish, hebrew","speaks_english, spanish, italian","speaks_english, spanish, italian (poorly)","speaks_english, spanish, other","speaks_english, spanish, portuguese, italian","speaks_english, spanish, tagalog, portuguese, japanese","speaks_english, swahili","speaks_english, swahili (poorly)","speaks_english, swedish","speaks_english, swedish (poorly)","speaks_english, tagalog","speaks_english, tagalog (fluently), german, spanish","speaks_english, tagalog (fluently), spanish (okay), farsi (poorly)","speaks_english, tagalog (okay)","speaks_english, tagalog (okay), spanish (poorly)","speaks_english, tagalog (poorly)","speaks_english, tagalog (poorly), c++ (okay)","speaks_english, tagalog, chinese","speaks_english, tagalog, spanish","speaks_english, tamil (poorly), spanish (poorly)","speaks_english, tamil, spanish","speaks_english, thai (okay)","speaks_english, thai, other","speaks_english, turkish (poorly), spanish (okay)","speaks_english, vietnamese","speaks_english, vietnamese (fluently)","speaks_english, vietnamese (fluently), sign language (poorly)","speaks_english, vietnamese (okay)","speaks_english, vietnamese (poorly)","speaks_english, yiddish (poorly)",status_available,status_married,status_seeing someone,status_single,status_unknown,sex
0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,2,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0,3,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7996,3,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
7997,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
7998,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
